In [5]:
import requests
import xml.etree.cElementTree as et
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdmn
from io import StringIO

In [20]:
flags_url = 'http://rating.chgk.info/tournaments.php?tournament_id={}&download_data=tournament_export_teams_with_flags&format=csv'
teams_on_synch_url = 'https://rating.chgk.info/synch.php?download_data=teams_synch_data&tournament_id={}'

In [7]:
tournament_id = 5965

In [46]:
# Download
teams_pd = pd.read_csv(StringIO(requests.get(teams_on_synch_url.format(tournament_id)).text), index_col=None,
                          sep=',', error_bad_lines=False, )
# Parse
teams_pd = pd.DataFrame({'id': teams_pd['ID представителя'],
                         'fio': teams_pd.iloc[:,2:5].apply(lambda x: ' '.join(x), axis=1),
                         'team_id': teams_pd['Команда'],
                         'team_name': teams_pd['ID команды'],
                         'city': teams_pd['Город'],
                         'flags': teams_pd['Флаги']})
# Flags: Общий, Вуз, Ш
teams_pd.loc[~teams_pd.flags.isin(['Вуз', 'Ш']), 'flags'] = 'Общий'
teams_pd.head()

,city,fio,flags,id,team_id,team_name
0,Аахен,Ализаде Джамиль Заур оглы,Общий,100822,3146,Авось
1,Аахен,Ализаде Джамиль Заур оглы,Общий,100822,71273,Кефир
2,Аахен,Ализаде Джамиль Заур оглы,Общий,100822,71275,С нами Лиза
3,Аахен,Ализаде Джамиль Заур оглы,Общий,100822,74447,Эквилибриум
4,Барнаул,Соломонов Иван Геннадьевич,Вуз,63377,53086,Ноунеймы


In [49]:
teams_pd_count = teams_pd.groupby(['id', 'flags']).count().iloc[:,0].reset_index().rename({'city':'count'}, axis=1)
teams_pd_count.head()

,id,flags,count
0,3645,Вуз,2
1,10732,Вуз,2
2,10732,Ш,1
3,12589,Вуз,2
4,12589,Общий,8


In [70]:
teams_pd_count_w = pd.pivot_table(teams_pd_count, values='count', index='id', columns='flags').fillna(0).astype(int).reset_index()
teams_pd_count_w = teams_pd[['id', 'fio', 'city']].drop_duplicates().merge(teams_pd_count_w, on='id', how='inner')
teams_pd_count_w.head()

,id,fio,city,Вуз,Общий,Ш
0,100822,Ализаде Джамиль Заур оглы,Аахен,0,4,0
1,63377,Соломонов Иван Геннадьевич,Барнаул,2,1,0
2,16459,Кривенко Сергей Николаевич,Витебск,0,1,0
3,26101,Прошин Алексей Евгеньевич,Воронеж,1,1,0
4,40904,Шишкин Александр Александрович,Екатеринбург,4,0,0


In [73]:
teams_pd_count_w.to_csv('../data/count_flags_{}.csv'.format(tournament_id), encoding='utf-8-sig')